<a href="https://colab.research.google.com/github/EngrAhmadMakki/ghf/blob/main/Solution_of_Simultaneous_Online_and_Offline_Machine_Scheduling_and_Robot_Coordination_Problems_by_Reinforcement_Learning_and%C2%A0Metaheuristics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Genetic Algorithm with Q-Learning in Python**
This implementation demonstrates a combination of Genetic Algorithm (GA) and Q-learning. The process involves evolving a population using crossover and mutation, while Q-learning is used to decide the best actions for optimizing the fitness of the population.


The figure below shows the **training procedure of Q-GA (Q-learning combined with Genetic Algorithm)**:

![Training procedure of Q-GA](/content/Q-GA.jpeg)



In [ ]:
# Import necessary libraries
import numpy as np
import random


## **Step 1: Parameter Initialization**

We initialize parameters for both the genetic algorithm and Q-learning.

- `population_size`: The number of individuals in the population.
- `mutation_rate`: The probability of a mutation occurring on a gene.
- `crossover_rate`: The probability of applying crossover between two parents.
- `generations`: Maximum number of iterations the genetic algorithm will run.
- `alpha`, `gamma`, and `epsilon`: Q-learning parameters (learning rate, discount factor, and exploration rate).


In [ ]:
# Parameters for the Genetic Algorithm
population_size = 20
mutation_rate = 0.01
crossover_rate = 0.7
generations = 100

# Q-learning parameters
alpha = 0.1  # learning rate
gamma = 0.9  # discount factor
epsilon = 0.1  # exploration rate


## **Step 2: Q-table Initialization**

We initialize the Q-table, which will store Q-values for two actions: crossover and mutation. The Q-table will have dimensions based on the population size and the number of actions.


In [ ]:
# Initialize the Q-table (actions are mutation and crossover rates)
actions = ['crossover', 'mutation']
Q_table = np.zeros((population_size, len(actions)))

## **Step 3: Fitness Function**

The fitness function evaluates how "fit" an individual is. For demonstration, we use a simple fitness function that sums the individual's binary genes. A higher sum indicates better fitness.


In [ ]:
# Define a simple fitness function
def fitness_function(individual):
    # For demonstration purposes, we use the sum of the genes as fitness
    return sum(individual)

## **Step 4: Population Initialization**

We generate the initial population, where each individual is represented by a list of binary values (`0` or `1`).


In [ ]:
# Initialize the population (binary encoded individuals)
def initialize_population(population_size, gene_length):
    return [np.random.randint(0, 2, gene_length).tolist() for _ in range(population_size)]

## **Step 5: Crossover and Mutation Operators**

We define two key genetic operations: crossover and mutation. These operations create new offspring from existing individuals in the population.

- `crossover`: Combines the genes of two parents.
- `mutation`: Flips bits in an individual's genes based on the mutation rate.

In [ ]:
# Crossover operation
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

# Mutation operation
def mutation(individual):
    for i in range(len(individual)):
        if random.random() < mutation_rate:
            individual[i] = 1 - individual[i]  # Flip the bit
    return individual

## **Step 6: Parent Selection (Tournament Selection)**

We select parents using tournament selection. A subset of individuals is chosen, and the one with the highest fitness is selected as a parent.


In [ ]:
# Select individuals based on fitness (tournament selection)
def select_individual(population, fitness_values):
    tournament_size = 3
    selected = random.sample(list(zip(population, fitness_values)), tournament_size)
    selected.sort(key=lambda x: x[1], reverse=True)
    return selected[0][0]

## **Step 7: Q-Learning Action Selection (ε-Greedy)**

The algorithm selects whether to apply crossover or mutation using an ε-greedy policy. With probability `epsilon`, it explores a random action, and otherwise, it exploits the best action learned so far.


In [ ]:
# Q-Learning action selection (ε-greedy)
def choose_action(state, epsilon):
    if random.random() < epsilon:
        return random.choice(actions)
    else:
        return actions[np.argmax(Q_table[state])]


## **Step 8: Q-table Update**

We update the Q-table based on the reward received from the new population's fitness. The Q-value update is based on the Q-learning formula.


In [ ]:
# Update Q-table
def update_Q_table(state, action_idx, reward, next_state):
    best_next_action = np.argmax(Q_table[next_state])
    Q_table[state, action_idx] = Q_table[state, action_idx] + alpha * (
        reward + gamma * Q_table[next_state, best_next_action] - Q_table[state, action_idx])


## **Step 9: Main Genetic Algorithm with Q-Learning Loop**

Here we integrate the genetic algorithm with Q-learning. The population evolves over a number of generations, and Q-learning is used to decide the best actions to improve the population's fitness.

In [ ]:
# Main genetic algorithm with Q-learning loop
def genetic_algorithm():
    population = initialize_population(population_size, gene_length=10)
    for generation in range(generations):
        fitness_values = [fitness_function(individual) for individual in population]

        new_population = []
        for _ in range(population_size // 2):  # Generating new population by crossover and mutation
            parent1 = select_individual(population, fitness_values)
            parent2 = select_individual(population, fitness_values)

            action = choose_action(_, epsilon)  # Choose action (crossover/mutation)
            if action == 'crossover':
                child1, child2 = crossover(parent1, parent2)
            else:  # Apply mutation
                child1, child2 = mutation(parent1), mutation(parent2)

            new_population.extend([child1, child2])

        # Evaluate new population fitness
        new_fitness_values = [fitness_function(individual) for individual in new_population]

        # Update Q-table based on the new population's fitness (reward is fitness gain)
        for i in range(population_size):
            reward = new_fitness_values[i] - fitness_values[i]
            update_Q_table(i, actions.index(action), reward, (i + 1) % population_size)

        # Check if maximum iteration is reached
        if generation == generations - 1:
            print("Reached final iteration")
            break

        # Update population for next generation
        population = new_population

    print("Q-table after training:")
    print(Q_table)

## **Step 10: Running the Genetic Algorithm**

Finally, we run the genetic algorithm to evolve the population over the defined number of generations and print the Q-table after training.


In [ ]:
# Running the Genetic Algorithm
genetic_algorithm()

Reached final iteration
Q-table after training:
[[0.16496673 0.05498821]
 [0.16294691 0.06242408]
 [0.16296386 0.07358122]
 [0.16655192 0.0859907 ]
 [0.171106   0.08036942]
 [0.17133276 0.06857716]
 [0.17400933 0.05129401]
 [0.17541885 0.06329365]
 [0.17503209 0.0722906 ]
 [0.17269268 0.07315901]
 [0.16841078 0.06293913]
 [0.16299244 0.0621853 ]
 [0.15873507 0.06973963]
 [0.15747673 0.07914056]
 [0.15885482 0.07818248]
 [0.1621142  0.07688919]
 [0.16588733 0.0781177 ]
 [0.16856325 0.07309005]
 [0.16871431 0.05075149]
 [0.16625572 0.05848406]]


In [ ]:
# # Import necessary libraries
# import numpy as np
# import random

# # Parameters for the Genetic Algorithm
# population_size = 20
# mutation_rate = 0.01
# crossover_rate = 0.7
# generations = 100

# # Q-learning parameters
# alpha = 0.1  # learning rate
# gamma = 0.9  # discount factor
# epsilon = 0.1  # exploration rate

# # Initialize the Q-table (actions are mutation and crossover rates)
# actions = ['crossover', 'mutation']
# Q_table = np.zeros((population_size, len(actions)))

# # Define a simple fitness function
# def fitness_function(individual):
#     # For demonstration purposes, we use the sum of the genes as fitness
#     return sum(individual)

# # Initialize the population (binary encoded individuals)
# def initialize_population(population_size, gene_length):
#     return [np.random.randint(0, 2, gene_length).tolist() for _ in range(population_size)]

# # Crossover operation
# def crossover(parent1, parent2):
#     point = random.randint(1, len(parent1) - 1)
#     child1 = parent1[:point] + parent2[point:]
#     child2 = parent2[:point] + parent1[point:]
#     return child1, child2

# # Mutation operation
# def mutation(individual):
#     for i in range(len(individual)):
#         if random.random() < mutation_rate:
#             individual[i] = 1 - individual[i]  # Flip the bit
#     return individual

# # Select individuals based on fitness (tournament selection)
# def select_individual(population, fitness_values):
#     tournament_size = 3
#     selected = random.sample(list(zip(population, fitness_values)), tournament_size)
#     selected.sort(key=lambda x: x[1], reverse=True)
#     return selected[0][0]

# # Q-Learning action selection (ε-greedy)
# def choose_action(state, epsilon):
#     if random.random() < epsilon:
#         return random.choice(actions)
#     else:
#         return actions[np.argmax(Q_table[state])]

# # Update Q-table
# def update_Q_table(state, action_idx, reward, next_state):
#     best_next_action = np.argmax(Q_table[next_state])
#     Q_table[state, action_idx] = Q_table[state, action_idx] + alpha * (
#         reward + gamma * Q_table[next_state, best_next_action] - Q_table[state, action_idx])

# # Main genetic algorithm with Q-learning loop
# def genetic_algorithm():
#     population = initialize_population(population_size, gene_length=10)
#     for generation in range(generations):
#         fitness_values = [fitness_function(individual) for individual in population]

#         new_population = []
#         for _ in range(population_size // 2):  # Generating new population by crossover and mutation
#             parent1 = select_individual(population, fitness_values)
#             parent2 = select_individual(population, fitness_values)

#             action = choose_action(_, epsilon)  # Choose action (crossover/mutation)
#             if action == 'crossover':
#                 child1, child2 = crossover(parent1, parent2)
#             else:  # Apply mutation
#                 child1, child2 = mutation(parent1), mutation(parent2)

#             new_population.extend([child1, child2])

#         # Evaluate new population fitness
#         new_fitness_values = [fitness_function(individual) for individual in new_population]

#         # Update Q-table based on the new population's fitness (reward is fitness gain)
#         for i in range(population_size):
#             reward = new_fitness_values[i] - fitness_values[i]
#             update_Q_table(i, actions.index(action), reward, (i + 1) % population_size)

#         # Check if maximum iteration is reached
#         if generation == generations - 1:
#             print("Reached final iteration")
#             break

#         # Update population for next generation
#         population = new_population

#     print("Q-table after training:")
#     print(Q_table)

# # Running the Genetic Algorithm
# genetic_algorithm()

# **Genetic Algorithm with Q-Learning: Four-Segment Chromosome Representation**

This Colab implements a hybrid Genetic Algorithm (GA) with Q-learning for solving machine scheduling problems, using a Four-Segment Chromosome Representation for optimization.

We will focus on:
1. Cell formation (machine grouping)
2. Machine selection (operation assignment)
3. Local sequencing (job sequencing within cells)
4. Global scheduling (overall scheduling of jobs and product families)


In [ ]:
# Import necessary libraries
import numpy as np
import random

# Define Genetic Algorithm Parameters
Here, we define key parameters for the Genetic Algorithm (GA) such as population size, mutation rate, and generations. Additionally, we define Q-learning parameters such as learning rate, discount factor, and exploration rate.


In [ ]:
# Parameters for the Genetic Algorithm
population_size = 20
mutation_rate = 0.01
crossover_rate = 0.7
generations = 100

# Q-learning parameters
alpha = 0.1  # learning rate
gamma = 0.9  # discount factor
epsilon = 0.1  # exploration rate

# Define Machines, Jobs, and Product Families
We define the machine resources (`m1`, `m2`, etc.) and the jobs that need to be scheduled. Additionally, we define product families, which contain jobs grouped together.


In [ ]:
# Machine and job definitions (as per the image)
machines = ['m1', 'm2', 'm3', 'm4', 'm5']
cells = ['c1', 'c2']  # Cell formation gene

# Individualized products (jobs j1 and j2)
jobs = ['j1', 'j2']
operations = {
    'j1': ['o11', 'o12', 'o13', 'o14'],
    'j2': ['o21', 'o22', 'o23']
}

# Product families (Family 1 with j3, Family 2 with j4)
product_families = {
    'j3': ['o31', 'o32'],  # Family 1
    'j4': ['o41', 'o42']   # Family 2
}

# Volume of products in each family
family_volumes = {
    'j3': 3,  # 3 products in Family 1
    'j4': 2   # 2 products in Family 2
}

# Define Fitness Function
The fitness function evaluates the schedule's efficiency. It aims to minimize the total makespan or another relevant scheduling metric.


In [ ]:
# # Fitness function (total makespan or any other scheduling metric)
# def fitness_function(schedule):
#     # Simplified fitness function, adjust for actual scheduling problem
#     return sum(sum(machine_schedule) for machine_schedule in schedule)


# Fitness function: calculate based on a mock schedule where numerical values are extracted
def fitness_function(individual):
    """
    Calculate the fitness of an individual based on the number of operations.
    This is a simplified fitness function and can be adjusted to your specific problem.
    """
    cell_assignment, machine_assignment, local_sequence, global_schedule = individual

    # Mock-up: Assign fitness based on the number of operations (just as an example)
    fitness = 0

    # Example: Fitness could be based on the number of operations assigned to machines
    for job in machine_assignment:
        fitness += len(machine_assignment[job])  # Add the number of operations

    # Optionally adjust based on other factors, like local sequencing or global schedule

    return fitness

# Initialize Population with Four-Segment Chromosome Representation
This section initializes the population of chromosomes using the four-segment representation. Each chromosome consists of:
1. **Cell formation**: Grouping machines into cells.
2. **Machine selection**: Assigning operations to machines.
3. **Local sequencing**: Ordering jobs and operations within machines and product families.
4. **Global scheduling**: Scheduling both jobs and product families.


In [ ]:
# Four-Segment Chromosome Representation (with product families)
def initialize_population(population_size, gene_length):
    population = []
    for _ in range(population_size):
        # 1st Segment: Cell formation (Assign machines to cells)
        cell_assignment = [random.choice(cells) for _ in range(len(machines))]

        # 2nd Segment: Machine selection (Assign operations to machines)
        machine_assignment = {job: [random.choice(machines) for _ in operations[job]] for job in jobs}
        machine_assignment.update({family: [random.choice(machines) for _ in product_families[family]] for family in product_families})

        # 3rd Segment: Local sequencing (Order jobs within machines and families)
        local_sequence = {job: random.sample(operations[job], len(operations[job])) for job in jobs}
        local_sequence.update({family: random.sample(product_families[family], len(product_families[family])) for family in product_families})

        # 4th Segment: Global scheduling (Overall schedule of jobs and families)
        global_schedule = random.sample(jobs + list(product_families.keys()), len(jobs) + len(product_families))  # Random order of jobs and families

        population.append((cell_assignment, machine_assignment, local_sequence, global_schedule))
    return population

# Crossover and Mutation Operations
The crossover and mutation operations are essential for genetic algorithms. The crossover exchanges segments between two parents to produce new offspring, while mutation introduces variability into the population.

In [ ]:
# Crossover operation
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1[0]) - 1)

    # Cross cells, machine assignments, local sequencing, and global scheduling
    child1 = (
        parent1[0][:point] + parent2[0][point:],
        parent1[1],  # Can perform more complex crossover for this segment
        parent1[2],
        parent1[3][:point] + parent2[3][point:]
    )
    child2 = (
        parent2[0][:point] + parent1[0][point:],
        parent2[1],
        parent2[2],
        parent2[3][:point] + parent1[3][point:]
    )
    return child1, child2

# Mutation operation
def mutation(individual):
    # Ensure individual is mutable (convert to list if necessary)
    individual = list(individual)

    # Mutate each segment randomly
    if random.random() < mutation_rate:
        individual[0] = [random.choice(cells) for _ in individual[0]]  # Mutate cell formation

    if random.random() < mutation_rate:
        individual[1] = {job: [random.choice(machines) for _ in operations[job]] for job in jobs}  # Mutate machine selection
        individual[1].update({family: [random.choice(machines) for _ in product_families[family]] for family in product_families})

    if random.random() < mutation_rate:
        individual[2] = {job: random.sample(operations[job], len(operations[job])) for job in jobs}  # Mutate local sequencing
        individual[2].update({family: random.sample(product_families[family], len(product_families[family])) for family in product_families})

    if random.random() < mutation_rate:
        individual[3] = random.sample(jobs + list(product_families.keys()), len(jobs) + len(product_families))  # Mutate global scheduling

    return individual

# Selection and Q-Learning Action Selection
Tournament selection is used to choose parents based on their fitness. Q-Learning helps select actions (crossover or mutation) based on the state of the population.


In [ ]:
# Select individuals based on fitness
def select_individual(population, fitness_values):
    tournament_size = 3
    selected = random.sample(list(zip(population, fitness_values)), tournament_size)
    selected.sort(key=lambda x: x[1], reverse=True)
    return selected[0][0]

# Define possible actions for Q-learning: crossover and mutation
actions = ['crossover', 'mutation']

# Q-Learning action selection (ε-greedy)
def choose_action(state, epsilon):
    if random.random() < epsilon:
        return random.choice(actions)
    else:
        return actions[np.argmax(Q_table[state])]


# Q-Table Update Function
The Q-table is updated based on the rewards (improvements in fitness) after applying actions (crossover or mutation). The goal is to learn the best strategy for evolving the population over generations.


In [ ]:
# # Update Q-table
# def update_Q_table(state, action_idx, reward, next_state):
#     best_next_action = np.argmax(Q_table[next_state])
#     Q_table[state, action_idx] = Q_table[state, action_idx] + alpha * (
#         reward + gamma * Q_table[next_state, best_next_action] - Q_table[state, action_idx])

# Update Q-table
def update_Q_table(state, action_idx, reward, next_state):
    best_next_action = np.argmax(Q_table[next_state])  # Find best next action
    Q_table[state, action_idx] = Q_table[state, action_idx] + alpha * (
        reward + gamma * Q_table[next_state, best_next_action] - Q_table[state, action_idx])


# Main Genetic Algorithm with Q-Learning Loop
This is the main loop where the genetic algorithm operates. It includes the initialization of the population, selection of parents, crossover, mutation, and updating the Q-table.


In [ ]:
# Update Q-table based on the fitness improvement
def genetic_algorithm():
    population = initialize_population(population_size, gene_length=10)
    for generation in range(generations):
        fitness_values = [fitness_function(individual) for individual in population]

        new_population = []
        for _ in range(population_size // 2):
            parent1 = select_individual(population, fitness_values)
            parent2 = select_individual(population, fitness_values)

            action = choose_action(_, epsilon)
            if action == 'crossover':
                child1, child2 = crossover(parent1, parent2)
            else:  # Apply mutation
                child1, child2 = mutation(parent1), mutation(parent2)

            new_population.extend([child1, child2])

        # Evaluate new population fitness
        new_fitness_values = [fitness_function(individual) for individual in new_population]

        # Update Q-table based on the new population's fitness
        for i in range(population_size):
            reward = new_fitness_values[i] - fitness_values[i]
            if reward > 0:
                reward = 1  # Positive reward for improvement
            elif reward < 0:
                reward = -1  # Negative reward for worse performance
            else:
                reward = 0  # No reward for no change

            update_Q_table(i, actions.index(action), reward, (i + 1) % population_size)

        # Update population for the next generation
        population = new_population

    print("Q-table after training:")
    print(Q_table)

# Run the Genetic Algorithm
Finally, we run the genetic algorithm with Q-learning to optimize the scheduling problem.


In [ ]:
# Running the Genetic Algorithm
genetic_algorithm()

Q-table after training:
[[0.07516176 0.07116486]
 [0.07627687 0.07362578]
 [0.07657874 0.07647516]
 [0.07307146 0.07579815]
 [0.07076778 0.07224609]
 [0.07017628 0.06970029]
 [0.06990116 0.06635273]
 [0.06995932 0.06898969]
 [0.0702843  0.07070528]
 [0.07080275 0.07098926]
 [0.07152912 0.06965009]
 [0.07250563 0.07038787]
 [0.07347678 0.07317597]
 [0.07306422 0.0746188 ]
 [0.07257117 0.0740927 ]
 [0.07220245 0.07349458]
 [0.0718152  0.07278486]
 [0.07233548 0.07220908]
 [0.07290222 0.06845176]
 [0.07357621 0.07094563]]


In [ ]:
import numpy as np
import random

# Parameters for the Genetic Algorithm
population_size = 20
mutation_rate = 0.01
crossover_rate = 0.7
generations = 100

# Q-learning parameters
alpha = 0.5  # Learning rate
gamma = 0.9  # Discount factor
epsilon = 0.2  # Exploration rate

# Define possible actions for Q-learning: crossover and mutation
actions = ['crossover', 'mutation']

# Machine and job definitions (as per the image)
machines = ['m1', 'm2', 'm3', 'm4', 'm5']
cells = ['c1', 'c2']  # Cell formation gene

# Individualized products (jobs j1 and j2)
jobs = ['j1', 'j2']
operations = {
    'j1': ['o11', 'o12', 'o13', 'o14'],
    'j2': ['o21', 'o22', 'o23']
}

# Product families (Family 1 with j3, Family 2 with j4)
product_families = {
    'j3': ['o31', 'o32'],  # Family 1
    'j4': ['o41', 'o42']   # Family 2
}

# Volume of products in each family
family_volumes = {
    'j3': 3,  # 3 products in Family 1
    'j4': 2   # 2 products in Family 2
}

# Initialize Q-table with zeros
Q_table = np.zeros((population_size, len(actions)))

# Fitness function: calculate based on a mock schedule where numerical values are extracted
def fitness_function(individual):
    """
    Calculate the fitness of an individual based on the number of operations.
    This is a simplified fitness function and can be adjusted to your specific problem.
    """
    cell_assignment, machine_assignment, local_sequence, global_schedule = individual
    fitness = 0

    # Fitness based on the number of operations assigned to machines
    for job in machine_assignment:
        fitness += len(machine_assignment[job])  # Add the number of operations

    # Example penalty for inefficiency or randomness
    penalty = random.uniform(0.1, 1)
    fitness -= penalty

    return fitness

# Four-Segment Chromosome Representation (with product families)
def initialize_population(population_size, gene_length):
    population = []
    for _ in range(population_size):
        # 1st Segment: Cell formation (Assign machines to cells)
        cell_assignment = [random.choice(cells) for _ in range(len(machines))]

        # 2nd Segment: Machine selection (Assign operations to machines)
        machine_assignment = {job: [random.choice(machines) for _ in operations[job]] for job in jobs}
        machine_assignment.update({family: [random.choice(machines) for _ in product_families[family]] for family in product_families})

        # 3rd Segment: Local sequencing (Order jobs within machines and families)
        local_sequence = {job: random.sample(operations[job], len(operations[job])) for job in jobs}
        local_sequence.update({family: random.sample(product_families[family], len(product_families[family])) for family in product_families})

        # 4th Segment: Global scheduling (Overall schedule of jobs and families)
        global_schedule = random.sample(jobs + list(product_families.keys()), len(jobs) + len(product_families))  # Random order of jobs and families

        population.append([cell_assignment, machine_assignment, local_sequence, global_schedule])
    return population

# Crossover operation
def crossover(parent1, parent2):
    point = random.randint(1, len(parent1[0]) - 1)

    # Cross cells, machine assignments, local sequencing, and global scheduling
    child1 = [
        parent1[0][:point] + parent2[0][point:],
        parent1[1],
        parent1[2],
        parent1[3][:point] + parent2[3][point:]
    ]
    child2 = [
        parent2[0][:point] + parent1[0][point:],
        parent2[1],
        parent2[2],
        parent2[3][:point] + parent1[3][point:]
    ]
    return child1, child2

# Mutation operation
def mutation(individual):
    individual = list(individual)

    # Mutate each segment randomly
    if random.random() < mutation_rate:
        individual[0] = [random.choice(cells) for _ in individual[0]]  # Mutate cell formation

    if random.random() < mutation_rate:
        individual[1] = {job: [random.choice(machines) for _ in operations[job]] for job in jobs}  # Mutate machine selection
        individual[1].update({family: [random.choice(machines) for _ in product_families[family]] for family in product_families})

    if random.random() < mutation_rate:
        individual[2] = {job: random.sample(operations[job], len(operations[job])) for job in jobs}  # Mutate local sequencing
        individual[2].update({family: random.sample(product_families[family], len(product_families[family])) for family in product_families})

    if random.random() < mutation_rate:
        individual[3] = random.sample(jobs + list(product_families.keys()), len(jobs) + len(product_families))  # Mutate global scheduling

    return individual

# Select individuals based on fitness
def select_individual(population, fitness_values):
    tournament_size = 3
    selected = random.sample(list(zip(population, fitness_values)), tournament_size)
    selected.sort(key=lambda x: x[1], reverse=True)
    return selected[0][0]

# Q-Learning action selection (ε-greedy)
def choose_action(state, epsilon):
    if random.random() < epsilon:
        return random.choice(actions)
    else:
        return actions[np.argmax(Q_table[state])]

# Update Q-table
def update_Q_table(state, action_idx, reward, next_state):
    best_next_action = np.argmax(Q_table[next_state])
    Q_table[state, action_idx] = Q_table[state, action_idx] + alpha * (
        reward + gamma * Q_table[next_state, best_next_action] - Q_table[state, action_idx])
    print(f"Q-value updated for state {state}, action {action_idx}: {Q_table[state, action_idx]}")

# Main genetic algorithm with Q-learning loop
def genetic_algorithm():
    population = initialize_population(population_size, gene_length=10)

    for generation in range(generations):
        fitness_values = [fitness_function(individual) for individual in population]

        new_population = []
        for i in range(population_size // 2):
            parent1 = select_individual(population, fitness_values)
            parent2 = select_individual(population, fitness_values)

            action = choose_action(i, epsilon)
            if action == 'crossover':
                child1, child2 = crossover(parent1, parent2)
            else:  # Apply mutation
                child1, child2 = mutation(parent1), mutation(parent2)

            new_population.extend([child1, child2])

        # Evaluate new population fitness
        new_fitness_values = [fitness_function(individual) for individual in new_population]

        # Update Q-table based on the new population's fitness (reward is fitness gain)
        for i in range(population_size):
            reward = new_fitness_values[i] - fitness_values[i]
            print(f"Fitness difference (reward) for individual {i}: {reward}")  # Check reward value
            if reward != 0:
                update_Q_table(i, actions.index(action), reward, (i + 1) % population_size)
                print(f"Updated Q-table after applying reward: {Q_table}")  # Check Q-table update

        # Update population for next generation
        population = new_population

    print("Q-table after training:")
    print(Q_table)

# Running the Genetic Algorithm
genetic_algorithm()

Streaming output truncated to the last 5000 lines.
 [0.78531746 1.2707827 ]
 [1.35996238 0.64438016]
 [0.88130166 0.50449038]
 [0.34921273 0.39071095]
 [0.69733767 0.82465977]
 [1.15445001 0.79089204]
 [0.98118291 1.31997075]]
Fitness difference (reward) for individual 14: -0.1959636442831858
Q-value updated for state 14, action 1: 0.6207940037719191
Updated Q-table after applying reward: [[0.98807398 0.8111562 ]
 [0.89013428 0.44439459]
 [0.40001472 0.24359102]
 [0.8328759  0.575921  ]
 [1.0470219  0.66770249]
 [0.63572156 0.89537196]
 [1.19513628 1.1743288 ]
 [1.55177956 0.56517208]
 [0.76212841 0.92426787]
 [0.59179922 0.41535285]
 [0.80138857 0.60964514]
 [0.99898273 0.96323139]
 [1.22303111 0.95645054]
 [0.78531746 1.2707827 ]
 [1.35996238 0.620794  ]
 [0.88130166 0.50449038]
 [0.34921273 0.39071095]
 [0.69733767 0.82465977]
 [1.15445001 0.79089204]
 [0.98118291 1.31997075]]
Fitness difference (reward) for individual 15: 0.18753973604827756
Q-value updated for state 15, action 1: 

# Flexible Job Shop Scheduling Problem (FJSP) Using a Genetic Algorithm (GA) Approach

In this notebook, we will implement the **Flexible Job Shop Scheduling Problem (FJSP)** using a **Genetic Algorithm (GA)** approach. This method will involve several key steps to define the problem instance, encoding the solutions, evaluating their fitness, and applying genetic operations like crossover and mutation.

## Problem Overview

In the **Flexible Job Shop Scheduling Problem (FJSP)**, we have multiple jobs, each consisting of a set of tasks. Each task can be processed on one or more available machines, but each machine can only process one task at a time. The objective is to schedule the tasks in such a way that the makespan (the total completion time) is minimized.

---

### Step 1: Define the Problem Instance

The problem instance will be represented as:

1. **Jobs**: A list of jobs, where each job consists of multiple tasks.
2. **Machines**: A set of machines, and each machine can process certain tasks. Machines are characterized by their processing times for each task.

#### Problem Representation
- **Jobs**: List of jobs, where each job contains a sequence of tasks.
- **Machines**: A list of machines, where each machine can process specific tasks with different processing times.

Let's represent this using a Python-like structure:

```python
# Number of jobs and machines
num_jobs = 5
num_machines = 3

# Example structure for tasks: (task_id, list_of_possible_machines)
tasks = [
    [(0, [0, 1]), (1, [0, 2])],  # Job 0: Task 0 on machine 0 or 1, Task 1 on machine 0 or 2
    [(0, [1, 2]), (1, [0, 2])],  # Job 1: Task 0 on ma


# Flexible Job Shop Scheduling Problem (FJSP) using a Genetic Algorithm (GA) Approach

The steps will involve the following:

## 1. Instance Storage Format
- Representing the problem (jobs, machines, and tasks).

## Encoding Scheme
- How we will encode a solution for the FJSP problem.

## Decoding/Encoding and Fitness Evaluation
- How we will evaluate the fitness of a solution (i.e., schedule) and encode/decode solutions.

## Crossover and Mutation
- Customizing these operations for the FJSP.


In [ ]:
import random

# Example of problem data (jobs, machines, and task durations)
jobs = [
    {"tasks": [5, 4]},  # Job 1: Task 1 takes 5 units of time, Task 2 takes 4 units
    {"tasks": [3, 6]},  # Job 2: Task 1 takes 3 units of time, Task 2 takes 6 units
    {"tasks": [7, 2]},  # Job 3: Task 1 takes 7 units of time, Task 2 takes 2 units
]

machines = ["M1", "M2", "M3"]  # List of machines

# Function to encode a solution (chromosome)
def encode_solution(num_jobs, num_machines):
    chromosome = []
    machine_end_times = {machine: 0 for machine in machines}  # Track machine availability

    for job_id in range(num_jobs):
        job_end_time = 0  # Track when the current job can start

        for task_id in range(len(jobs[job_id]["tasks"])):
            # Randomly assign a machine for the task
            machine_id = random.choice(machines)

            # Find the earliest time this task can start based on the machine's availability and job dependencies
            start_time = max(machine_end_times[machine_id], job_end_time)

            task_duration = jobs[job_id]["tasks"][task_id]
            end_time = start_time + task_duration  # When does this task finish?

            # Update machine's next available time
            machine_end_times[machine_id] = end_time

            # Update job end time to reflect the completion of this task
            job_end_time = max(job_end_time, end_time)

            # Add to the chromosome
            chromosome.append((job_id, task_id, machine_id, start_time))

    return chromosome


# Fitness function: Minimize makespan, tardiness, and machine penalty
def calculate_fitness(chromosome, num_jobs, num_machines, job_due_dates=None):
    machine_end_times = {machine: 0 for machine in machines}  # Track machine end times
    job_end_times = [0] * num_jobs  # Track job completion times

    total_tardiness = 0  # Initialize tardiness counter

    for job_id, task_id, machine_id, start_time in chromosome:
        task_duration = jobs[job_id]["tasks"][task_id]
        end_time = start_time + task_duration  # When does this task finish?

        # Update job end time (the time when the job is done)
        job_end_times[job_id] = max(job_end_times[job_id], end_time)

        # Update machine's available time
        machine_end_times[machine_id] = max(machine_end_times[machine_id], end_time)

    # Calculate makespan: time when last job is completed
    makespan = max(job_end_times)

    # Penalize tardiness (if there are due dates for the jobs)
    if job_due_dates:
        for job_id in range(num_jobs):
            tardiness = max(0, job_end_times[job_id] - job_due_dates[job_id])
            total_tardiness += tardiness

    # Penalize for underutilized machines (based on idle time)
    machine_penalty = sum(max(0, machine_end_times[machine_id] - makespan) for machine_id in machines)

    # Combine makespan, tardiness, and machine penalties
    fitness_score = makespan + total_tardiness + machine_penalty
    return fitness_score

# Crossover: Perform one-point crossover
def crossover(parent1, parent2):
    # Perform one-point crossover (cut at a random point)
    crossover_point = random.randint(1, len(parent1) - 1)

    # Create child chromosomes by swapping part of the parents
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]

    return child1, child2

# Mutation: Randomly reassign tasks to different machines
def mutate(chromosome, max_retries=10):
    """ Mutates the chromosome and ensures no overlapping tasks on the same machine. """

    for _ in range(max_retries):
        # Select two random tasks from different jobs
        idx1, idx2 = random.sample(range(len(chromosome)), 2)

        # Extract the tasks
        job1, task1, machine1, start_time1 = chromosome[idx1]
        job2, task2, machine2, start_time2 = chromosome[idx2]

        # Swap machines between two tasks
        new_machine1 = random.choice(machines)
        new_machine2 = random.choice(machines)

        # Update the chromosome with new machine assignments
        chromosome[idx1] = (job1, task1, new_machine1, start_time1)
        chromosome[idx2] = (job2, task2, new_machine2, start_time2)

        # Check if the updated schedule is valid
        if is_valid_schedule(chromosome):
            return chromosome  # Return valid mutation

    # If mutation fails after max_retries, apply a more flexible strategy
    print("Mutation failed after maximum retries, attempting flexible mutation.")
    return flexible_mutation(chromosome)

def flexible_mutation(chromosome):
    """ Attempt to adjust the schedule to make a valid mutation when the original one fails. """

    # Try to swap machines or adjust start times for the tasks
    machine_end_times = {machine: 0 for machine in machines}

    # Iterate through the chromosome and adjust tasks one by one
    for i, (job_id, task_id, machine_id, start_time) in enumerate(chromosome):
        task_duration = jobs[job_id]["tasks"][task_id]

        # Try to adjust the start time if necessary (avoid overlaps)
        if machine_end_times[machine_id] > start_time:
            # Adjust the start time to avoid overlap
            new_start_time = machine_end_times[machine_id]
            chromosome[i] = (job_id, task_id, machine_id, new_start_time)

        # Update machine's availability
        machine_end_times[machine_id] = max(machine_end_times[machine_id], start_time + task_duration)

    # Return the adjusted chromosome
    return chromosome


# Tournament selection: Select parents for crossover
def select_parents(population, fitness, tournament_size=5):
    parents = []
    population_size = len(population)

    # Ensure tournament size is not greater than population size
    tournament_size = min(tournament_size, population_size)

    for _ in range(len(population) // 2):
        # Randomly select tournament_size individuals
        tournament = random.sample(list(zip(population, fitness)), tournament_size)
        # Choose the best individual from the tournament
        winner = min(tournament, key=lambda x: x[1])
        parents.append(winner[0])

    return parents


# Main genetic algorithm function
def genetic_algorithm(num_jobs, num_machines, population_size=50, generations=100, job_due_dates=None):
    # Initialize population with random solutions
    population = [encode_solution(num_jobs, num_machines) for _ in range(population_size)]

    for generation in range(generations):
        # Evaluate fitness for the entire population
        fitness = [calculate_fitness(chromosome, num_jobs, num_machines, job_due_dates) for chromosome in population]

        # Selection (Tournament Selection)
        selected_parents = select_parents(population, fitness)

        # Crossover to generate offspring
        offspring = []
        for i in range(0, len(selected_parents) - 1, 2):  # Ensure we always have pairs
            parent1, parent2 = selected_parents[i], selected_parents[i + 1]
            child1, child2 = crossover(parent1, parent2)
            offspring.extend([child1, child2])

        # If the number of parents is odd, randomly select one parent for direct copying to the next generation
        if len(selected_parents) % 2 != 0:
            offspring.append(selected_parents[-1])  # Directly add the last parent to the offspring

        # Mutation
        mutated_offspring = [mutate(child) for child in offspring]

        # Make sure the population is not empty
        if not mutated_offspring:
            print(f"Error: Population is empty after mutation in generation {generation + 1}")
            break

        # Update population
        population = mutated_offspring

        # Store best solution (minimizing fitness score)
        best_solution = min(population, key=lambda x: calculate_fitness(x, num_jobs, num_machines, job_due_dates))
        print(f"Generation {generation + 1}, Best Makespan: {calculate_fitness(best_solution, num_jobs, num_machines, job_due_dates)}")

    return best_solution





# Example usage of the genetic algorithm

# Due dates for jobs (optional, can be None if not needed)
job_due_dates = [20, 18, 15]  # Jobs 1, 2, and 3 should be completed by these times.

# Run the genetic algorithm
best_solution = genetic_algorithm(num_jobs=3, num_machines=3, population_size=50, generations=100, job_due_dates=job_due_dates)

# Decode and print the best solution (chromosome)
timeline = decode_chromosome(best_solution)
print("Best Schedule (Timeline):")
for job_id, tasks in timeline.items():
    print(f"Job {job_id + 1}:")
    for task_id, machine_id, start_time in tasks:
        print(f"  Task {task_id + 1} on {machine_id} starts at {start_time}")

NameError: name 'is_valid_schedule' is not defined

# Correct version from Professor "Handle my comments, in the attached file; they are marked with "XXX". This should avoid the population running empty.

# Flexible Job Shop Scheduling Problem (FJSP) using a Genetic Algorithm (GA) Approach

In [ ]:
import random

# Example problem data
jobs = [
    {"tasks": [5, 4]},  # Job 1: Task 1 takes 5 units, Task 2 takes 4 units
    {"tasks": [3, 6]},  # Job 2: Task 1 takes 3 units, Task 2 takes 6 units
    {"tasks": [7, 2]},  # Job 3: Task 1 takes 7 units, Task 2 takes 2 units
]

machines = ["M1", "M2", "M3"]  # List of machines

# Function to encode a solution
def encode_solution(num_jobs, num_machines):
    chromosome = []
    machine_end_times = {machine: 0 for machine in machines}  # Track machine availability

    for job_id in range(num_jobs):
        job_end_time = 0  # Track job completion

        for task_id in range(len(jobs[job_id]["tasks"])):
            # Assign to the machine with the earliest availability
            compatible_machines = machines
            machine_id = min(compatible_machines, key=lambda m: machine_end_times[m])

            start_time = max(machine_end_times[machine_id], job_end_time)
            task_duration = jobs[job_id]["tasks"][task_id]
            end_time = start_time + task_duration

            machine_end_times[machine_id] = end_time
            job_end_time = end_time

            chromosome.append((job_id, task_id, machine_id, start_time))

    return chromosome

# Fitness function
def calculate_fitness(chromosome, num_jobs, num_machines, job_due_dates=None):
    machine_end_times = {machine: 0 for machine in machines}
    job_end_times = [0] * num_jobs
    total_tardiness = 0

    for job_id, task_id, machine_id, start_time in chromosome:
        task_duration = jobs[job_id]["tasks"][task_id]
        end_time = start_time + task_duration
        job_end_times[job_id] = max(job_end_times[job_id], end_time)
        machine_end_times[machine_id] = max(machine_end_times[machine_id], end_time)

    makespan = max(job_end_times)
    if job_due_dates:
        total_tardiness = sum(max(0, job_end_times[j] - job_due_dates[j]) for j in range(num_jobs))

    return makespan + total_tardiness

# Helper function to check schedule validity
def is_valid_schedule(chromosome):
    machine_end_times = {machine: 0 for machine in machines}
    for _, _, machine_id, start_time in chromosome:
        if start_time < machine_end_times[machine_id]:
            return False
        machine_end_times[machine_id] = start_time
    return True

# Decode chromosome into a readable schedule
def decode_chromosome(chromosome):
    timeline = {}
    for job_id, task_id, machine_id, start_time in chromosome:
        if job_id not in timeline:
            timeline[job_id] = []
        timeline[job_id].append((task_id, machine_id, start_time))
    return timeline

# Crossover function
def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return repair_solution(child1), repair_solution(child2)

# Repair function to ensure valid schedule
def repair_solution(chromosome):
    machine_end_times = {machine: 0 for machine in machines}
    repaired_chromosome = []
    for job_id, task_id, machine_id, start_time in chromosome:
        compatible_machines = machines
        machine_id = min(compatible_machines, key=lambda m: machine_end_times[m])
        start_time = machine_end_times[machine_id]
        task_duration = jobs[job_id]["tasks"][task_id]
        end_time = start_time + task_duration
        machine_end_times[machine_id] = end_time
        repaired_chromosome.append((job_id, task_id, machine_id, start_time))
    return repaired_chromosome

# Mutation function
def mutate(chromosome):
    idx = random.randint(0, len(chromosome) - 1)
    job_id, task_id, _, start_time = chromosome[idx]
    new_machine_id = random.choice(machines)
    chromosome[idx] = (job_id, task_id, new_machine_id, start_time)
    return repair_solution(chromosome)

# Genetic algorithm
def genetic_algorithm(num_jobs, num_machines, population_size=50, generations=100, job_due_dates=None):
    population = [encode_solution(num_jobs, num_machines) for _ in range(population_size)]

    for generation in range(generations):
        fitness = [calculate_fitness(chromosome, num_jobs, num_machines, job_due_dates) for chromosome in population]
        selected_parents = random.choices(population, k=len(population))
        offspring = []
        for i in range(0, len(selected_parents) - 1, 2):
            child1, child2 = crossover(selected_parents[i], selected_parents[i + 1])
            offspring.extend([mutate(child1), mutate(child2)])
        population = offspring
        best_solution = min(population, key=lambda x: calculate_fitness(x, num_jobs, num_machines, job_due_dates))
        print(f"Generation {generation + 1}, Best Fitness: {calculate_fitness(best_solution, num_jobs, num_machines, job_due_dates)}")
    return best_solution

# Example usage
job_due_dates = [20, 18, 15]
best_solution = genetic_algorithm(num_jobs=3, num_machines=3, population_size=50, generations=100, job_due_dates=job_due_dates)
timeline = decode_chromosome(best_solution)
print("Best Schedule (Timeline):")
for job_id, tasks in timeline.items():
    print(f"Job {job_id + 1}: {tasks}")


Generation 1, Best Fitness: 11
Generation 2, Best Fitness: 11
Generation 3, Best Fitness: 11
Generation 4, Best Fitness: 11
Generation 5, Best Fitness: 11
Generation 6, Best Fitness: 11
Generation 7, Best Fitness: 11
Generation 8, Best Fitness: 11
Generation 9, Best Fitness: 11
Generation 10, Best Fitness: 11
Generation 11, Best Fitness: 11
Generation 12, Best Fitness: 11
Generation 13, Best Fitness: 11
Generation 14, Best Fitness: 11
Generation 15, Best Fitness: 11
Generation 16, Best Fitness: 11
Generation 17, Best Fitness: 11
Generation 18, Best Fitness: 11
Generation 19, Best Fitness: 11
Generation 20, Best Fitness: 11
Generation 21, Best Fitness: 11
Generation 22, Best Fitness: 11
Generation 23, Best Fitness: 11
Generation 24, Best Fitness: 11
Generation 25, Best Fitness: 11
Generation 26, Best Fitness: 11
Generation 27, Best Fitness: 11
Generation 28, Best Fitness: 11
Generation 29, Best Fitness: 11
Generation 30, Best Fitness: 11
Generation 31, Best Fitness: 11
Generation 32, Be

# Extend FJSP code with the Q-learning used in the Cheng et al. Paper.

In [ ]:
import random
import numpy as np

# Q-learning parameters
epsilon = 0.1  # Exploration rate
alpha = 0.1    # Learning rate
gamma = 0.9    # Discount factor
q_table = {}   # Q-table for storing state-action values

# Define actions: 0 = crossover, 1 = mutation
actions = [0, 1]

# Initialize Q-table
def initialize_q_table(states, actions):
    for state in states:
        q_table[state] = {action: 0 for action in actions}

# Define state representation
def get_state(population, fitness):
    # Example state: fitness improvement trend
    best_fitness = min(fitness)
    avg_fitness = sum(fitness) / len(fitness)
    return (round(best_fitness, 2), round(avg_fitness, 2))

# Select action using epsilon-greedy policy
def select_action(state):
    if random.random() < epsilon:  # Explore
        return random.choice(actions)
    # Exploit: Select the best action for the given state
    return max(q_table[state], key=q_table[state].get)

# Update Q-table
def update_q_table(state, action, reward, next_state):
    max_next_q = max(q_table[next_state].values())
    q_table[state][action] = q_table[state][action] + alpha * (reward + gamma * max_next_q - q_table[state][action])

# Reward function
def calculate_reward(previous_fitness, current_fitness):
    return previous_fitness - current_fitness  # Positive reward for fitness improvement

# Genetic algorithm with Q-learning
def genetic_algorithm_q_learning(num_jobs, num_machines, population_size=50, generations=100, job_due_dates=None):
    # Initialize population
    population = [encode_solution(num_jobs, num_machines) for _ in range(population_size)]
    fitness = [calculate_fitness(chromosome, num_jobs, num_machines, job_due_dates) for chromosome in population]
    states = {get_state(population, fitness)}  # Initialize state set
    initialize_q_table(states, actions)

    for generation in range(generations):
        # Calculate current state
        state = get_state(population, fitness)

        # Select action using epsilon-greedy policy
        action = select_action(state)

        # Apply action: crossover or mutation
        if action == 0:  # Apply crossover
            offspring = []
            selected_parents = random.choices(population, k=len(population))
            for i in range(0, len(selected_parents) - 1, 2):
                child1, child2 = crossover(selected_parents[i], selected_parents[i + 1])
                offspring.extend([child1, child2])
            new_population = offspring
        elif action == 1:  # Apply mutation
            new_population = [mutate(individual) for individual in population]

        # Ensure population remains feasible
        new_population = [repair_solution(individual) for individual in new_population]

        # Evaluate new fitness
        new_fitness = [calculate_fitness(chromosome, num_jobs, num_machines, job_due_dates) for chromosome in new_population]

        # Calculate reward
        reward = calculate_reward(min(fitness), min(new_fitness))

        # Update Q-table
        next_state = get_state(new_population, new_fitness)
        if next_state not in q_table:
            q_table[next_state] = {action: 0 for action in actions}
        update_q_table(state, action, reward, next_state)

        # Update population and fitness
        population = new_population
        fitness = new_fitness

        # Print progress
        best_solution = min(population, key=lambda x: calculate_fitness(x, num_jobs, num_machines, job_due_dates))
        print(f"Generation {generation + 1}, Best Fitness: {calculate_fitness(best_solution, num_jobs, num_machines, job_due_dates)}")

    return best_solution

# Example usage
job_due_dates = [20, 18, 15]
best_solution = genetic_algorithm_q_learning(num_jobs=3, num_machines=3, population_size=50, generations=100, job_due_dates=job_due_dates)
timeline = decode_chromosome(best_solution)
print("Best Schedule (Timeline):")
for job_id, tasks in timeline.items():
    print(f"Job {job_id + 1}: {tasks}")


Generation 1, Best Fitness: 11
Generation 2, Best Fitness: 11
Generation 3, Best Fitness: 11
Generation 4, Best Fitness: 11
Generation 5, Best Fitness: 11
Generation 6, Best Fitness: 11
Generation 7, Best Fitness: 11
Generation 8, Best Fitness: 11
Generation 9, Best Fitness: 11
Generation 10, Best Fitness: 11
Generation 11, Best Fitness: 11
Generation 12, Best Fitness: 11
Generation 13, Best Fitness: 11
Generation 14, Best Fitness: 11
Generation 15, Best Fitness: 11
Generation 16, Best Fitness: 11
Generation 17, Best Fitness: 11
Generation 18, Best Fitness: 11
Generation 19, Best Fitness: 11
Generation 20, Best Fitness: 11
Generation 21, Best Fitness: 11
Generation 22, Best Fitness: 11
Generation 23, Best Fitness: 11
Generation 24, Best Fitness: 11
Generation 25, Best Fitness: 11
Generation 26, Best Fitness: 11
Generation 27, Best Fitness: 11
Generation 28, Best Fitness: 11
Generation 29, Best Fitness: 11
Generation 30, Best Fitness: 11
Generation 31, Best Fitness: 11
Generation 32, Be

# Extended version of the code transport times between machines

In [ ]:
import random
import numpy as np

# Example problem data
jobs = [
    {"tasks": [5, 4]},  # Job 1: Task 1 takes 5 units, Task 2 takes 4 units
    {"tasks": [3, 6]},  # Job 2: Task 1 takes 3 units, Task 2 takes 6 units
    {"tasks": [7, 2]},  # Job 3: Task 1 takes 7 units, Task 2 takes 2 units
]

machines = ["M1", "M2", "M3"]  # List of machines

# Transport times between machines (constant for simplicity)
transport_times = {
    ("M1", "M2"): 2,
    ("M1", "M3"): 3,
    ("M2", "M1"): 2,
    ("M2", "M3"): 1,
    ("M3", "M1"): 3,
    ("M3", "M2"): 1,
    ("M1", "M1"): 0,
    ("M2", "M2"): 0,
    ("M3", "M3"): 0,
}

# Q-learning parameters
epsilon = 0.1  # Exploration rate
alpha = 0.1    # Learning rate
gamma = 0.9    # Discount factor
q_table = {}   # Q-table for storing state-action values

actions = [0, 1]  # 0 = crossover, 1 = mutation

# Initialize Q-table
def initialize_q_table(states, actions):
    for state in states:
        q_table[state] = {action: 0 for action in actions}

# Define state representation
def get_state(population, fitness):
    best_fitness = min(fitness)
    avg_fitness = sum(fitness) / len(fitness)
    return (round(best_fitness, 2), round(avg_fitness, 2))

# Select action using epsilon-greedy policy
def select_action(state):
    if random.random() < epsilon:
        return random.choice(actions)
    return max(q_table[state], key=q_table[state].get)

# Update Q-table
def update_q_table(state, action, reward, next_state):
    max_next_q = max(q_table[next_state].values())
    q_table[state][action] = q_table[state][action] + alpha * (reward + gamma * max_next_q - q_table[state][action])

# Reward function
def calculate_reward(previous_fitness, current_fitness):
    return max(0, previous_fitness - current_fitness)  # Positive reward for fitness improvement

# Encode solution
def encode_solution(num_jobs, num_machines):
    chromosome = []
    machine_end_times = {machine: 0 for machine in machines}
    for job_id in range(num_jobs):
        job_end_time = 0
        previous_machine = None
        for task_id in range(len(jobs[job_id]["tasks"])):
            compatible_machines = machines
            machine_id = min(compatible_machines, key=lambda m: machine_end_times[m])
            transport_time = transport_times.get((previous_machine, machine_id), 0) if previous_machine else 0
            start_time = max(machine_end_times[machine_id], job_end_time + transport_time)
            task_duration = jobs[job_id]["tasks"][task_id]
            end_time = start_time + task_duration
            machine_end_times[machine_id] = end_time
            job_end_time = end_time
            previous_machine = machine_id
            chromosome.append((job_id, task_id, machine_id, start_time))
    return chromosome

# Fitness function
def calculate_fitness(chromosome, num_jobs, num_machines, job_due_dates=None):
    machine_end_times = {machine: 0 for machine in machines}
    job_end_times = [0] * num_jobs
    total_tardiness = 0
    for i, (job_id, task_id, machine_id, start_time) in enumerate(chromosome):
        task_duration = jobs[job_id]["tasks"][task_id]
        transport_time = 0
        if i > 0:
            _, _, prev_machine, _ = chromosome[i - 1]
            transport_time = transport_times.get((prev_machine, machine_id), 0)
        end_time = start_time + task_duration + transport_time
        job_end_times[job_id] = max(job_end_times[job_id], end_time)
        machine_end_times[machine_id] = max(machine_end_times[machine_id], end_time)
    makespan = max(job_end_times)
    if job_due_dates:
        total_tardiness = sum(max(0, job_end_times[j] - job_due_dates[j]) for j in range(num_jobs))
    return makespan + total_tardiness

# Repair solution
def repair_solution(chromosome):
    machine_end_times = {machine: 0 for machine in machines}
    previous_machine = None
    repaired_chromosome = []
    for job_id, task_id, machine_id, start_time in chromosome:
        transport_time = transport_times.get((previous_machine, machine_id), 0) if previous_machine else 0
        start_time = max(machine_end_times[machine_id], start_time + transport_time)
        task_duration = jobs[job_id]["tasks"][task_id]
        end_time = start_time + task_duration
        machine_end_times[machine_id] = end_time
        previous_machine = machine_id
        repaired_chromosome.append((job_id, task_id, machine_id, start_time))
    return repaired_chromosome

# Genetic algorithm
def genetic_algorithm(num_jobs, num_machines, population_size=50, generations=100, job_due_dates=None):
    population = [encode_solution(num_jobs, num_machines) for _ in range(population_size)]
    fitness = [calculate_fitness(chromosome, num_jobs, num_machines, job_due_dates) for chromosome in population]
    states = {get_state(population, fitness)}
    initialize_q_table(states, actions)
    for generation in range(generations):
        state = get_state(population, fitness)
        action = select_action(state)
        if action == 0:
            offspring = []
            selected_parents = random.choices(population, k=len(population))
            for i in range(0, len(selected_parents) - 1, 2):
                child1, child2 = crossover(selected_parents[i], selected_parents[i + 1])
                offspring.extend([child1, child2])
            new_population = offspring
        elif action == 1:
            new_population = [mutate(individual) for individual in population]
        new_population = [repair_solution(individual) for individual in new_population]
        new_fitness = [calculate_fitness(chromosome, num_jobs, num_machines, job_due_dates) for chromosome in new_population]
        reward = calculate_reward(min(fitness), min(new_fitness))
        next_state = get_state(new_population, new_fitness)
        if next_state not in q_table:
            q_table[next_state] = {action: 0 for action in actions}
        update_q_table(state, action, reward, next_state)
        population = new_population
        fitness = new_fitness
        best_solution = min(population, key=lambda x: calculate_fitness(x, num_jobs, num_machines, job_due_dates))
        print(f"Generation {generation + 1}, Best Fitness: {calculate_fitness(best_solution, num_jobs, num_machines, job_due_dates)}")
    return best_solution

# Example usage
job_due_dates = [20, 18, 15]
best_solution = genetic_algorithm(num_jobs=3, num_machines=3, population_size=50, generations=100, job_due_dates=job_due_dates)
print("Best Schedule (Timeline):")
timeline = decode_chromosome(best_solution)
for job_id, tasks in timeline.items():
    print(f"Job {job_id + 1}: {tasks}")


Generation 1, Best Fitness: 37
Generation 2, Best Fitness: 50
Generation 3, Best Fitness: 66
Generation 4, Best Fitness: 82
Generation 5, Best Fitness: 99
Generation 6, Best Fitness: 99
Generation 7, Best Fitness: 105
Generation 8, Best Fitness: 111
Generation 9, Best Fitness: 143
Generation 10, Best Fitness: 165
Generation 11, Best Fitness: 181
Generation 12, Best Fitness: 185
Generation 13, Best Fitness: 209
Generation 14, Best Fitness: 229
Generation 15, Best Fitness: 237
Generation 16, Best Fitness: 267
Generation 17, Best Fitness: 258
Generation 18, Best Fitness: 272
Generation 19, Best Fitness: 296
Generation 20, Best Fitness: 330
Generation 21, Best Fitness: 348
Generation 22, Best Fitness: 348
Generation 23, Best Fitness: 367
Generation 24, Best Fitness: 392
Generation 25, Best Fitness: 416
Generation 26, Best Fitness: 424
Generation 27, Best Fitness: 424
Generation 28, Best Fitness: 444
Generation 29, Best Fitness: 446
Generation 30, Best Fitness: 446
Generation 31, Best Fitne

In [ ]:
import random

# Example problem data
jobs = [
    {"tasks": [5, 4]},  # Job 1: Task 1 takes 5 units, Task 2 takes 4 units
    {"tasks": [3, 6]},  # Job 2: Task 1 takes 3 units, Task 2 takes 6 units
    {"tasks": [7, 2]},  # Job 3: Task 1 takes 7 units, Task 2 takes 2 units
]

machines = ["M1", "M2", "M3"]  # List of machines

# Transport times between machines (constant for simplicity)
transport_times = {
    ("M1", "M2"): 2,
    ("M1", "M3"): 3,
    ("M2", "M1"): 2,
    ("M2", "M3"): 1,
    ("M3", "M1"): 3,
    ("M3", "M2"): 1,
    ("M1", "M1"): 0,
    ("M2", "M2"): 0,
    ("M3", "M3"): 0,
}

# Encode solution
def encode_solution(num_jobs, num_machines):
    chromosome = []
    machine_end_times = {machine: 0 for machine in machines}
    for job_id in range(num_jobs):
        job_end_time = 0
        previous_machine = None
        for task_id in range(len(jobs[job_id]["tasks"])):
            machine_id = min(machines, key=lambda m: machine_end_times[m])
            transport_time = transport_times.get((previous_machine, machine_id), 0) if previous_machine else 0
            start_time = max(machine_end_times[machine_id], job_end_time + transport_time)
            task_duration = jobs[job_id]["tasks"][task_id]
            end_time = start_time + task_duration
            machine_end_times[machine_id] = end_time
            job_end_time = end_time
            previous_machine = machine_id
            chromosome.append((job_id, task_id, machine_id, start_time))
    return chromosome

# Fitness function
def calculate_fitness(chromosome):
    machine_end_times = {machine: 0 for machine in machines}
    job_end_times = [0] * len(jobs)
    for i, (job_id, task_id, machine_id, start_time) in enumerate(chromosome):
        task_duration = jobs[job_id]["tasks"][task_id]
        transport_time = 0
        if i > 0:
            _, _, prev_machine, _ = chromosome[i - 1]
            transport_time = transport_times.get((prev_machine, machine_id), 0)
        end_time = start_time + task_duration + transport_time
        job_end_times[job_id] = max(job_end_times[job_id], end_time)
        machine_end_times[machine_id] = max(machine_end_times[machine_id], end_time)
    makespan = max(job_end_times)
    return makespan

# Repair solution
def repair_solution(chromosome):
    repaired_chromosome = []
    machine_end_times = {machine: 0 for machine in machines}
    previous_machine = None
    for job_id, task_id, machine_id, start_time in chromosome:
        transport_time = transport_times.get((previous_machine, machine_id), 0) if previous_machine else 0
        start_time = max(machine_end_times[machine_id], start_time + transport_time)
        task_duration = jobs[job_id]["tasks"][task_id]
        end_time = start_time + task_duration
        machine_end_times[machine_id] = end_time
        previous_machine = machine_id
        repaired_chromosome.append((job_id, task_id, machine_id, start_time))
    return repaired_chromosome

# Genetic algorithm
def genetic_algorithm(num_jobs, num_machines, population_size=50, generations=100):
    population = [encode_solution(num_jobs, num_machines) for _ in range(population_size)]
    for generation in range(generations):
        fitness_values = [calculate_fitness(ind) for ind in population]
        best_fitness = min(fitness_values)
        best_solution = population[fitness_values.index(best_fitness)]
        print(f"Generation {generation + 1}, Best Fitness: {best_fitness}")

        # Selection: Tournament selection
        selected_parents = random.choices(population, k=len(population))

        # Crossover
        offspring = []
        for i in range(0, len(selected_parents) - 1, 2):
            parent1, parent2 = selected_parents[i], selected_parents[i + 1]
            crossover_point = random.randint(1, len(parent1) - 1)
            child1 = parent1[:crossover_point] + parent2[crossover_point:]
            child2 = parent2[:crossover_point] + parent1[crossover_point:]
            offspring.extend([repair_solution(child1), repair_solution(child2)])

        # Mutation
        for i in range(len(offspring)):
            if random.random() < 0.1:  # Mutation probability
                idx = random.randint(0, len(offspring[i]) - 1)
                job_id, task_id, _, start_time = offspring[i][idx]
                new_machine = random.choice(machines)
                offspring[i][idx] = (job_id, task_id, new_machine, start_time)
                offspring[i] = repair_solution(offspring[i])

        # Update population
        population = offspring

    return best_solution

# Example usage
best_solution = genetic_algorithm(num_jobs=3, num_machines=3, population_size=50, generations=100)
print("Best Schedule (Timeline):")
for job_id, task_id, machine_id, start_time in best_solution:
    print(f"Job {job_id + 1}, Task {task_id + 1} on {machine_id} starts at {start_time}")


Generation 1, Best Fitness: 20
Generation 2, Best Fitness: 23
Generation 3, Best Fitness: 24
Generation 4, Best Fitness: 28
Generation 5, Best Fitness: 29
Generation 6, Best Fitness: 30
Generation 7, Best Fitness: 31
Generation 8, Best Fitness: 33
Generation 9, Best Fitness: 38
Generation 10, Best Fitness: 39
Generation 11, Best Fitness: 40
Generation 12, Best Fitness: 41
Generation 13, Best Fitness: 46
Generation 14, Best Fitness: 47
Generation 15, Best Fitness: 53
Generation 16, Best Fitness: 54
Generation 17, Best Fitness: 55
Generation 18, Best Fitness: 56
Generation 19, Best Fitness: 57
Generation 20, Best Fitness: 58
Generation 21, Best Fitness: 74
Generation 22, Best Fitness: 73
Generation 23, Best Fitness: 74
Generation 24, Best Fitness: 74
Generation 25, Best Fitness: 74
Generation 26, Best Fitness: 79
Generation 27, Best Fitness: 76
Generation 28, Best Fitness: 93
Generation 29, Best Fitness: 82
Generation 30, Best Fitness: 93
Generation 31, Best Fitness: 101
Generation 32, B

# Task 1: Implement a new repair_solution function
- Create a list or array for each job containing the next task that can be scheduled (all predecessor tasks must already be scheduled).
- Initialize the list with the first task of each job.
- Iterate over the list and:
    - Find the task with the longest processing time.
    - Schedule this task on a suitable machine.
    - Update the machine's end time accordingly.
- Update the list:
- Remove the task just scheduled.
- Repeat until all tasks are scheduled.
---

In [ ]:
def repair_solution(jobs, machines):
    # Initialize the list with the first task of each job
    tasks_to_schedule = [{"job_id": i, "task_id": 0, "processing_time": job["tasks"][0]}
                         for i, job in enumerate(jobs)]
    machine_end_times = {machine: 0 for machine in machines}  # Track machine availability
    schedule = []  # Final schedule

    while tasks_to_schedule:
        # Find the task with the longest processing time
        task_to_schedule = max(tasks_to_schedule, key=lambda x: x["processing_time"])
        tasks_to_schedule.remove(task_to_schedule)

        # Determine the job and task
        job_id = task_to_schedule["job_id"]
        task_id = task_to_schedule["task_id"]

        # Schedule this task on the first available machine
        machine = min(machines, key=lambda m: machine_end_times[m])
        start_time = machine_end_times[machine]  # Machine's current end time
        processing_time = task_to_schedule["processing_time"]
        end_time = start_time + processing_time

        # Update the machine's end time
        machine_end_times[machine] = end_time

        # Add the scheduled task to the final schedule
        schedule.append({"job_id": job_id, "task_id": task_id, "machine": machine, "start_time": start_time})

        # Add the next task of the same job to the list if available
        if task_id + 1 < len(jobs[job_id]["tasks"]):
            tasks_to_schedule.append({
                "job_id": job_id,
                "task_id": task_id + 1,
                "processing_time": jobs[job_id]["tasks"][task_id + 1]
            })

    return schedule

In [ ]:
import random

# Example problem data
jobs = [
    {"tasks": [5, 4]},  # Job 1: Task 1 takes 5 units, Task 2 takes 4 units
    {"tasks": [3, 6]},  # Job 2: Task 1 takes 3 units, Task 2 takes 6 units
    {"tasks": [7, 2]},  # Job 3: Task 1 takes 7 units, Task 2 takes 2 units
]

machines = ["M1", "M2", "M3"]  # List of machines

# Transport times between machines (constant for simplicity)
transport_times = {
    ("M1", "M2"): 2,
    ("M1", "M3"): 3,
    ("M2", "M1"): 2,
    ("M2", "M3"): 1,
    ("M3", "M1"): 3,
    ("M3", "M2"): 1,
    ("M1", "M1"): 0,
    ("M2", "M2"): 0,
    ("M3", "M3"): 0,
}

# Encode solution
def encode_solution(num_jobs, num_machines):
    chromosome = []
    machine_end_times = {machine: 0 for machine in machines}
    for job_id in range(num_jobs):
        job_end_time = 0
        previous_machine = None
        for task_id in range(len(jobs[job_id]["tasks"])):
            machine_id = min(machines, key=lambda m: machine_end_times[m])
            transport_time = transport_times.get((previous_machine, machine_id), 0) if previous_machine else 0
            start_time = max(machine_end_times[machine_id], job_end_time + transport_time)
            task_duration = jobs[job_id]["tasks"][task_id]
            end_time = start_time + task_duration
            machine_end_times[machine_id] = end_time
            job_end_time = end_time
            previous_machine = machine_id
            chromosome.append((job_id, task_id, machine_id, start_time))
    return chromosome

# Fitness function
def calculate_fitness(chromosome):
    machine_end_times = {machine: 0 for machine in machines}
    job_end_times = [0] * len(jobs)
    for i, (job_id, task_id, machine_id, start_time) in enumerate(chromosome):
        job_id = int(job_id)  # Ensure job_id is an integer
        task_id = int(task_id)  # Ensure task_id is an integer
        task_duration = jobs[job_id]["tasks"][task_id]
        transport_time = 0
        if i > 0:
            _, _, prev_machine, _ = chromosome[i - 1]
            transport_time = transport_times.get((prev_machine, machine_id), 0)
        end_time = start_time + task_duration + transport_time
        job_end_times[job_id] = max(job_end_times[job_id], end_time)
        machine_end_times[machine_id] = max(machine_end_times[machine_id], end_time)
    makespan = max(job_end_times)
    return makespan

# Repair solution
def repair_solution(jobs, machines):
    tasks_to_schedule = [{"job_id": i, "task_id": 0, "processing_time": job["tasks"][0]}
                         for i, job in enumerate(jobs)]
    machine_end_times = {machine: 0 for machine in machines}
    schedule = []

    while tasks_to_schedule:
        task_to_schedule = max(tasks_to_schedule, key=lambda x: x["processing_time"])
        tasks_to_schedule.remove(task_to_schedule)

        job_id = task_to_schedule["job_id"]
        task_id = task_to_schedule["task_id"]

        machine = min(machines, key=lambda m: machine_end_times[m])
        start_time = machine_end_times[machine]
        processing_time = task_to_schedule["processing_time"]
        end_time = start_time + processing_time

        machine_end_times[machine] = end_time
        schedule.append((job_id, task_id, machine, start_time))

        if task_id + 1 < len(jobs[job_id]["tasks"]):
            tasks_to_schedule.append({
                "job_id": job_id,
                "task_id": task_id + 1,
                "processing_time": jobs[job_id]["tasks"][task_id + 1]
            })

    return schedule

# Genetic algorithm
def genetic_algorithm(num_jobs, num_machines, population_size=50, generations=100):
    population = [encode_solution(num_jobs, num_machines) for _ in range(population_size)]
    for generation in range(generations):
        fitness_values = [calculate_fitness(ind) for ind in population]
        best_fitness = min(fitness_values)
        best_solution = population[fitness_values.index(best_fitness)]
        print(f"Generation {generation + 1}, Best Fitness: {best_fitness}")

        # Selection: Tournament selection
        selected_parents = random.choices(population, k=len(population))

        # Crossover
        offspring = []
        for i in range(0, len(selected_parents) - 1, 2):
            parent1, parent2 = selected_parents[i], selected_parents[i + 1]
            crossover_point = random.randint(1, len(parent1) - 1)
            child1 = parent1[:crossover_point] + parent2[crossover_point:]
            child2 = parent2[:crossover_point] + parent1[crossover_point:]
            offspring.extend([repair_solution(jobs, machines), repair_solution(jobs, machines)])

        # Mutation
        for i in range(len(offspring)):
            if random.random() < 0.1:  # Mutation probability
                idx = random.randint(0, len(offspring[i]) - 1)
                job_id, task_id, _, start_time = offspring[i][idx]
                new_machine = random.choice(machines)
                offspring[i][idx] = (int(job_id), int(task_id), new_machine, start_time)
                offspring[i] = repair_solution(jobs, machines)

        # Update population
        population = offspring

    return best_solution

# Example usage
best_solution = genetic_algorithm(num_jobs=3, num_machines=3, population_size=50, generations=100)
print("Best Schedule (Timeline):")
for job_id, task_id, machine_id, start_time in best_solution:
    print(f"Job {job_id + 1}, Task {task_id + 1} on {machine_id} starts at {start_time}")

Generation 1, Best Fitness: 20
Generation 2, Best Fitness: 12
Generation 3, Best Fitness: 12
Generation 4, Best Fitness: 12
Generation 5, Best Fitness: 12
Generation 6, Best Fitness: 12
Generation 7, Best Fitness: 12
Generation 8, Best Fitness: 12
Generation 9, Best Fitness: 12
Generation 10, Best Fitness: 12
Generation 11, Best Fitness: 12
Generation 12, Best Fitness: 12
Generation 13, Best Fitness: 12
Generation 14, Best Fitness: 12
Generation 15, Best Fitness: 12
Generation 16, Best Fitness: 12
Generation 17, Best Fitness: 12
Generation 18, Best Fitness: 12
Generation 19, Best Fitness: 12
Generation 20, Best Fitness: 12
Generation 21, Best Fitness: 12
Generation 22, Best Fitness: 12
Generation 23, Best Fitness: 12
Generation 24, Best Fitness: 12
Generation 25, Best Fitness: 12
Generation 26, Best Fitness: 12
Generation 27, Best Fitness: 12
Generation 28, Best Fitness: 12
Generation 29, Best Fitness: 12
Generation 30, Best Fitness: 12
Generation 31, Best Fitness: 12
Generation 32, Be

# Task 2: Modify the `is_valid_schedule` function

- Change the function to calculate penalty values for schedule violations:
  - Identify violations where `start_time < machine_end_times[machine_id]`.
  - Compute the total time units of violations.
  - Multiply the total violation time by a penalty factor (e.g., 5 or 10).

- Add this penalty value to the fitness value in `calculate_fitness`
  - Include the penalty in the fitness calculation to better reflect the quality of the schedule.

In [ ]:
import random

# Example problem data
jobs = [
    {"tasks": [5, 4]},  # Job 1: Task 1 takes 5 units, Task 2 takes 4 units
    {"tasks": [3, 6]},  # Job 2: Task 1 takes 3 units, Task 2 takes 6 units
    {"tasks": [7, 2]},  # Job 3: Task 1 takes 7 units, Task 2 takes 2 units
]

machines = ["M1", "M2", "M3"]  # List of machines

# Transport times between machines (constant for simplicity)
transport_times = {
    ("M1", "M2"): 2,
    ("M1", "M3"): 3,
    ("M2", "M1"): 2,
    ("M2", "M3"): 1,
    ("M3", "M1"): 3,
    ("M3", "M2"): 1,
    ("M1", "M1"): 0,
    ("M2", "M2"): 0,
    ("M3", "M3"): 0,
}

# Encode solution
def encode_solution(num_jobs, num_machines):
    chromosome = []
    machine_end_times = {machine: 0 for machine in machines}
    for job_id in range(num_jobs):
        job_end_time = 0
        previous_machine = None
        for task_id in range(len(jobs[job_id]["tasks"])):
            machine_id = random.choice(machines)  # Introduce randomness for diversity
            transport_time = transport_times.get((previous_machine, machine_id), 0) if previous_machine else 0
            start_time = max(machine_end_times[machine_id], job_end_time + transport_time)
            task_duration = jobs[job_id]["tasks"][task_id]
            end_time = start_time + task_duration
            machine_end_times[machine_id] = end_time
            job_end_time = end_time
            previous_machine = machine_id
            chromosome.append((job_id, task_id, machine_id, start_time))
    return chromosome

# Fitness function
def calculate_fitness(chromosome):
    machine_end_times = {machine: 0 for machine in machines}
    job_end_times = [0] * len(jobs)
    penalty = 0

    for i, (job_id, task_id, machine_id, start_time) in enumerate(chromosome):
        job_id = int(job_id)  # Ensure job_id is an integer
        task_id = int(task_id)  # Ensure task_id is an integer
        task_duration = jobs[job_id]["tasks"][task_id]
        if start_time < machine_end_times[machine_id]:
            penalty += (machine_end_times[machine_id] - start_time) * 10

        transport_time = 0
        if i > 0:
            _, _, prev_machine, _ = chromosome[i - 1]
            transport_time = transport_times.get((prev_machine, machine_id), 0)
        end_time = start_time + task_duration + transport_time
        job_end_times[job_id] = max(job_end_times[job_id], end_time)
        machine_end_times[machine_id] = max(machine_end_times[machine_id], end_time)

    makespan = max(job_end_times)
    fitness = makespan + penalty * 0.1  # Scale down penalty
    return fitness

# Repair solution
def repair_solution(jobs, machines):
    tasks_to_schedule = [{"job_id": i, "task_id": 0, "processing_time": job["tasks"][0]}
                         for i, job in enumerate(jobs)]
    machine_end_times = {machine: 0 for machine in machines}
    schedule = []

    while tasks_to_schedule:
        task_to_schedule = max(tasks_to_schedule, key=lambda x: x["processing_time"])
        tasks_to_schedule.remove(task_to_schedule)

        job_id = task_to_schedule["job_id"]
        task_id = task_to_schedule["task_id"]

        machine = min(machines, key=lambda m: machine_end_times[m])
        start_time = machine_end_times[machine]
        processing_time = task_to_schedule["processing_time"]
        end_time = start_time + processing_time

        machine_end_times[machine] = end_time
        schedule.append((job_id, task_id, machine, start_time))

        if task_id + 1 < len(jobs[job_id]["tasks"]):
            tasks_to_schedule.append({
                "job_id": job_id,
                "task_id": task_id + 1,
                "processing_time": jobs[job_id]["tasks"][task_id + 1]
            })

    return schedule

# Genetic algorithm
def genetic_algorithm(num_jobs, num_machines, population_size=50, generations=100):
    population = [encode_solution(num_jobs, num_machines) for _ in range(population_size)]
    mutation_rate = 0.1  # Initial mutation rate

    for generation in range(generations):
        fitness_values = [calculate_fitness(ind) for ind in population]
        best_fitness = min(fitness_values)
        best_solution = population[fitness_values.index(best_fitness)]
        print(f"Generation {generation + 1}, Best Fitness: {best_fitness}")

        # Selection: Tournament selection
        selected_parents = random.choices(population, k=len(population))

        # Crossover
        offspring = []
        for i in range(0, len(selected_parents) - 1, 2):
            parent1, parent2 = selected_parents[i], selected_parents[i + 1]
            crossover_point1 = random.randint(1, len(parent1) // 2)
            crossover_point2 = random.randint(len(parent1) // 2, len(parent1) - 1)
            child1 = parent1[:crossover_point1] + parent2[crossover_point1:crossover_point2] + parent1[crossover_point2:]
            child2 = parent2[:crossover_point1] + parent1[crossover_point1:crossover_point2] + parent2[crossover_point2:]
            offspring.extend([repair_solution(jobs, machines), repair_solution(jobs, machines)])

        # Mutation
        for i in range(len(offspring)):
            if random.random() < mutation_rate:  # Dynamic mutation rate
                idx = random.randint(0, len(offspring[i]) - 1)
                job_id, task_id, _, start_time = offspring[i][idx]
                new_machine = random.choice(machines)
                offspring[i][idx] = (int(job_id), int(task_id), new_machine, start_time)
                offspring[i] = repair_solution(jobs, machines)

        # Update population
        population = offspring

        # Adjust mutation rate dynamically
        mutation_rate = max(0.05, mutation_rate * 0.99)

    return best_solution

# Example usage
best_solution = genetic_algorithm(num_jobs=3, num_machines=3, population_size=50, generations=100)
print("Best Schedule (Timeline):")
for job_id, task_id, machine_id, start_time in best_solution:
    print(f"Job {job_id + 1}, Task {task_id + 1} on {machine_id} starts at {start_time}")


Generation 1, Best Fitness: 16.0
Generation 2, Best Fitness: 15.0
Generation 3, Best Fitness: 15.0
Generation 4, Best Fitness: 15.0
Generation 5, Best Fitness: 15.0
Generation 6, Best Fitness: 15.0
Generation 7, Best Fitness: 15.0
Generation 8, Best Fitness: 15.0
Generation 9, Best Fitness: 15.0
Generation 10, Best Fitness: 15.0
Generation 11, Best Fitness: 15.0
Generation 12, Best Fitness: 15.0
Generation 13, Best Fitness: 15.0
Generation 14, Best Fitness: 15.0
Generation 15, Best Fitness: 15.0
Generation 16, Best Fitness: 15.0
Generation 17, Best Fitness: 15.0
Generation 18, Best Fitness: 15.0
Generation 19, Best Fitness: 15.0
Generation 20, Best Fitness: 15.0
Generation 21, Best Fitness: 15.0
Generation 22, Best Fitness: 15.0
Generation 23, Best Fitness: 15.0
Generation 24, Best Fitness: 15.0
Generation 25, Best Fitness: 15.0
Generation 26, Best Fitness: 15.0
Generation 27, Best Fitness: 15.0
Generation 28, Best Fitness: 15.0
Generation 29, Best Fitness: 15.0
Generation 30, Best Fit

# Task 3: Create larger instances for testing
- Generate at least 10 problem instances.
- Use 10 machines, 20 jobs, and 5 tasks per job for each instance.

In [ ]:
import random

# Example problem data generator
def generate_problem_instance(num_jobs, num_machines, num_tasks_per_job):
    jobs = [{"tasks": [random.randint(1, 10) for _ in range(num_tasks_per_job)]} for _ in range(num_jobs)]
    machines = [f"M{i+1}" for i in range(num_machines)]
    return jobs, machines

# Generate larger problem instances
problem_instances = [generate_problem_instance(20, 10, 5) for _ in range(10)]

# Transport times between machines (constant for simplicity)
transport_times = {
    (f"M{i+1}", f"M{j+1}"): random.randint(1, 5) if i != j else 0
    for i in range(10) for j in range(10)
}

# Encode solution
def encode_solution(jobs, machines):
    chromosome = []
    machine_end_times = {machine: 0 for machine in machines}
    for job_id in range(len(jobs)):
        job_end_time = 0
        previous_machine = None
        for task_id in range(len(jobs[job_id]["tasks"])):
            machine_id = random.choice(machines)  # Introduce randomness for diversity
            transport_time = transport_times.get((previous_machine, machine_id), 0) if previous_machine else 0
            start_time = max(machine_end_times[machine_id], job_end_time + transport_time)
            task_duration = jobs[job_id]["tasks"][task_id]
            end_time = start_time + task_duration
            machine_end_times[machine_id] = end_time
            job_end_time = end_time
            previous_machine = machine_id
            chromosome.append((job_id, task_id, machine_id, start_time))
    return chromosome

# Fitness function
def calculate_fitness(chromosome, jobs, machines):
    machine_end_times = {machine: 0 for machine in machines}
    job_end_times = [0] * len(jobs)
    penalty = 0

    for i, (job_id, task_id, machine_id, start_time) in enumerate(chromosome):
        job_id = int(job_id)  # Ensure job_id is an integer
        task_id = int(task_id)  # Ensure task_id is an integer
        task_duration = jobs[job_id]["tasks"][task_id]
        if start_time < machine_end_times[machine_id]:
            penalty += (machine_end_times[machine_id] - start_time) * 10

        transport_time = 0
        if i > 0:
            _, _, prev_machine, _ = chromosome[i - 1]
            transport_time = transport_times.get((prev_machine, machine_id), 0)
        end_time = start_time + task_duration + transport_time
        job_end_times[job_id] = max(job_end_times[job_id], end_time)
        machine_end_times[machine_id] = max(machine_end_times[machine_id], end_time)

    makespan = max(job_end_times)
    fitness = makespan + penalty * 0.1  # Scale down penalty
    return fitness

# Repair solution
def repair_solution(jobs, machines):
    tasks_to_schedule = [{"job_id": i, "task_id": 0, "processing_time": job["tasks"][0]}
                         for i, job in enumerate(jobs)]
    machine_end_times = {machine: 0 for machine in machines}
    schedule = []

    while tasks_to_schedule:
        task_to_schedule = max(tasks_to_schedule, key=lambda x: x["processing_time"])
        tasks_to_schedule.remove(task_to_schedule)

        job_id = task_to_schedule["job_id"]
        task_id = task_to_schedule["task_id"]

        machine = min(machines, key=lambda m: machine_end_times[m])
        start_time = machine_end_times[machine]
        processing_time = task_to_schedule["processing_time"]
        end_time = start_time + processing_time

        machine_end_times[machine] = end_time
        schedule.append((job_id, task_id, machine, start_time))

        if task_id + 1 < len(jobs[job_id]["tasks"]):
            tasks_to_schedule.append({
                "job_id": job_id,
                "task_id": task_id + 1,
                "processing_time": jobs[job_id]["tasks"][task_id + 1]
            })

    return schedule

# Genetic algorithm
def genetic_algorithm(jobs, machines, population_size=50, generations=100):
    population = [encode_solution(jobs, machines) for _ in range(population_size)]
    mutation_rate = 0.1  # Initial mutation rate

    for generation in range(generations):
        fitness_values = [calculate_fitness(ind, jobs, machines) for ind in population]
        best_fitness = min(fitness_values)
        best_solution = population[fitness_values.index(best_fitness)]
        print(f"Generation {generation + 1}, Best Fitness: {best_fitness}")

        # Selection: Tournament selection
        selected_parents = random.choices(population, k=len(population))

        # Crossover
        offspring = []
        for i in range(0, len(selected_parents) - 1, 2):
            parent1, parent2 = selected_parents[i], selected_parents[i + 1]
            crossover_point1 = random.randint(1, len(parent1) // 2)
            crossover_point2 = random.randint(len(parent1) // 2, len(parent1) - 1)
            child1 = parent1[:crossover_point1] + parent2[crossover_point1:crossover_point2] + parent1[crossover_point2:]
            child2 = parent2[:crossover_point1] + parent1[crossover_point1:crossover_point2] + parent2[crossover_point2:]
            offspring.extend([repair_solution(jobs, machines), repair_solution(jobs, machines)])

        # Mutation
        for i in range(len(offspring)):
            if random.random() < mutation_rate:  # Dynamic mutation rate
                idx = random.randint(0, len(offspring[i]) - 1)
                job_id, task_id, _, start_time = offspring[i][idx]
                new_machine = random.choice(machines)
                offspring[i][idx] = (int(job_id), int(task_id), new_machine, start_time)
                offspring[i] = repair_solution(jobs, machines)

        # Update population
        population = offspring

        # Adjust mutation rate dynamically
        mutation_rate = max(0.05, mutation_rate * 0.99)

    return best_solution

# Example usage for problem instances
for idx, (jobs, machines) in enumerate(problem_instances):
    print(f"Testing Problem Instance {idx + 1}")
    best_solution = genetic_algorithm(jobs, machines, population_size=50, generations=100)
    print("Best Schedule (Timeline):")
    for job_id, task_id, machine_id, start_time in best_solution:
        print(f"Job {job_id + 1}, Task {task_id + 1} on {machine_id} starts at {start_time}")
    print()

Testing Problem Instance 1
Generation 1, Best Fitness: 383.0
Generation 2, Best Fitness: 331.0
Generation 3, Best Fitness: 331.0
Generation 4, Best Fitness: 331.0
Generation 5, Best Fitness: 331.0
Generation 6, Best Fitness: 331.0
Generation 7, Best Fitness: 331.0
Generation 8, Best Fitness: 331.0
Generation 9, Best Fitness: 331.0
Generation 10, Best Fitness: 331.0
Generation 11, Best Fitness: 331.0
Generation 12, Best Fitness: 331.0
Generation 13, Best Fitness: 331.0
Generation 14, Best Fitness: 331.0
Generation 15, Best Fitness: 331.0
Generation 16, Best Fitness: 331.0
Generation 17, Best Fitness: 331.0
Generation 18, Best Fitness: 331.0
Generation 19, Best Fitness: 331.0
Generation 20, Best Fitness: 331.0
Generation 21, Best Fitness: 331.0
Generation 22, Best Fitness: 331.0
Generation 23, Best Fitness: 331.0
Generation 24, Best Fitness: 331.0
Generation 25, Best Fitness: 331.0
Generation 26, Best Fitness: 331.0
Generation 27, Best Fitness: 331.0
Generation 28, Best Fitness: 331.0
Ge

# Task 4(a): Test the Genetic Algorithm (GA) without Q-Learning.

In [ ]:
import random

# Example problem data generator
def generate_problem_instance(num_jobs, num_machines, num_tasks_per_job):
    jobs = [{"tasks": [random.randint(1, 10) for _ in range(num_tasks_per_job)]} for _ in range(num_jobs)]
    machines = [f"M{i+1}" for i in range(num_machines)]
    return jobs, machines

# Generate larger problem instances
problem_instances = [generate_problem_instance(20, 10, 5) for _ in range(10)]

# Transport times between machines (constant for simplicity)
transport_times = {
    (f"M{i+1}", f"M{j+1}"): random.randint(1, 5) if i != j else 0
    for i in range(10) for j in range(10)
}

# Encode solution
def encode_solution(jobs, machines):
    chromosome = []
    machine_end_times = {machine: 0 for machine in machines}
    for job_id in range(len(jobs)):
        job_end_time = 0
        previous_machine = None
        for task_id in range(len(jobs[job_id]["tasks"])):
            machine_id = random.choice(machines)  # Introduce randomness for diversity
            transport_time = transport_times.get((previous_machine, machine_id), 0) if previous_machine else 0
            start_time = max(machine_end_times[machine_id], job_end_time + transport_time)
            task_duration = jobs[job_id]["tasks"][task_id]
            end_time = start_time + task_duration
            machine_end_times[machine_id] = end_time
            job_end_time = end_time
            previous_machine = machine_id
            chromosome.append((job_id, task_id, machine_id, start_time))
    return chromosome

# Fitness function
def calculate_fitness(chromosome, jobs, machines):
    machine_end_times = {machine: 0 for machine in machines}
    job_end_times = [0] * len(jobs)
    penalty = 0

    for i, (job_id, task_id, machine_id, start_time) in enumerate(chromosome):
        job_id = int(job_id)  # Ensure job_id is an integer
        task_id = int(task_id)  # Ensure task_id is an integer
        task_duration = jobs[job_id]["tasks"][task_id]
        if start_time < machine_end_times[machine_id]:
            penalty += (machine_end_times[machine_id] - start_time) * 10

        transport_time = 0
        if i > 0:
            _, _, prev_machine, _ = chromosome[i - 1]
            transport_time = transport_times.get((prev_machine, machine_id), 0)
        end_time = start_time + task_duration + transport_time
        job_end_times[job_id] = max(job_end_times[job_id], end_time)
        machine_end_times[machine_id] = max(machine_end_times[machine_id], end_time)

    makespan = max(job_end_times)
    fitness = makespan + penalty * 0.1  # Scale down penalty
    return fitness

# Repair solution
def repair_solution(jobs, machines):
    tasks_to_schedule = [{"job_id": i, "task_id": 0, "processing_time": job["tasks"][0]}
                         for i, job in enumerate(jobs)]
    machine_end_times = {machine: 0 for machine in machines}
    schedule = []

    while tasks_to_schedule:
        task_to_schedule = max(tasks_to_schedule, key=lambda x: x["processing_time"])
        tasks_to_schedule.remove(task_to_schedule)

        job_id = task_to_schedule["job_id"]
        task_id = task_to_schedule["task_id"]

        machine = min(machines, key=lambda m: machine_end_times[m])
        start_time = machine_end_times[machine]
        processing_time = task_to_schedule["processing_time"]
        end_time = start_time + processing_time

        machine_end_times[machine] = end_time
        schedule.append((job_id, task_id, machine, start_time))

        if task_id + 1 < len(jobs[job_id]["tasks"]):
            tasks_to_schedule.append({
                "job_id": job_id,
                "task_id": task_id + 1,
                "processing_time": jobs[job_id]["tasks"][task_id + 1]
            })

    return schedule

# Genetic algorithm
def genetic_algorithm(jobs, machines, population_size=50, generations=100):
    population = [encode_solution(jobs, machines) for _ in range(population_size)]
    mutation_rate = 0.2  # Higher initial mutation rate to promote exploration

    for generation in range(generations):
        fitness_values = [calculate_fitness(ind, jobs, machines) for ind in population]
        best_fitness = min(fitness_values)
        best_solution = population[fitness_values.index(best_fitness)]
        average_fitness = sum(fitness_values) / len(fitness_values)
        print(f"Generation {generation + 1}: Best Fitness = {best_fitness}, Average Fitness = {average_fitness:.2f}")

        # Selection: Tournament selection
        selected_parents = random.choices(population, k=len(population))

        # Crossover
        offspring = []
        for i in range(0, len(selected_parents) - 1, 2):
            parent1, parent2 = selected_parents[i], selected_parents[i + 1]
            crossover_point1 = random.randint(1, len(parent1) // 2)
            crossover_point2 = random.randint(len(parent1) // 2, len(parent1) - 1)
            child1 = parent1[:crossover_point1] + parent2[crossover_point1:crossover_point2] + parent1[crossover_point2:]
            child2 = parent2[:crossover_point1] + parent1[crossover_point1:crossover_point2] + parent2[crossover_point2:]
            offspring.append(repair_solution(jobs, machines))
            offspring.append(repair_solution(jobs, machines))

        # Mutation
        for i in range(len(offspring)):
            if random.random() < mutation_rate:  # Dynamic mutation rate
                idx = random.randint(0, len(offspring[i]) - 1)
                job_id, task_id, _, start_time = offspring[i][idx]
                new_machine = random.choice(machines)
                offspring[i][idx] = (int(job_id), int(task_id), new_machine, start_time)
                offspring[i] = repair_solution(jobs, machines)

        # Update population
        population = offspring

        # Adjust mutation rate dynamically
        mutation_rate = max(0.05, mutation_rate * 0.95)  # Gradually reduce mutation rate

    return best_solution

# Example usage for problem instances
for idx, (jobs, machines) in enumerate(problem_instances):
    print(f"\nTesting Problem Instance {idx + 1}")
    best_solution = genetic_algorithm(jobs, machines, population_size=50, generations=100)
    print("Best Schedule (Timeline):")
    for job_id, task_id, machine_id, start_time in best_solution:
        print(f"Job {job_id + 1}, Task {task_id + 1} on {machine_id} starts at {start_time}")



Testing Problem Instance 1
Generation 1: Best Fitness = 422.0, Average Fitness = 519.72
Generation 2: Best Fitness = 347.0, Average Fitness = 347.00
Generation 3: Best Fitness = 347.0, Average Fitness = 347.00
Generation 4: Best Fitness = 347.0, Average Fitness = 347.00
Generation 5: Best Fitness = 347.0, Average Fitness = 347.00
Generation 6: Best Fitness = 347.0, Average Fitness = 347.00
Generation 7: Best Fitness = 347.0, Average Fitness = 347.00
Generation 8: Best Fitness = 347.0, Average Fitness = 347.00
Generation 9: Best Fitness = 347.0, Average Fitness = 347.00
Generation 10: Best Fitness = 347.0, Average Fitness = 347.00
Generation 11: Best Fitness = 347.0, Average Fitness = 347.00
Generation 12: Best Fitness = 347.0, Average Fitness = 347.00
Generation 13: Best Fitness = 347.0, Average Fitness = 347.00
Generation 14: Best Fitness = 347.0, Average Fitness = 347.00
Generation 15: Best Fitness = 347.0, Average Fitness = 347.00
Generation 16: Best Fitness = 347.0, Average Fitnes

# Task 4(b): Test the Genetic Algorithm (GA) with Q-Learning.

In [ ]:
import random
import numpy as np

# Example problem data generator
def generate_problem_instance(num_jobs, num_machines, num_tasks_per_job):
    jobs = [{"tasks": [random.randint(1, 10) for _ in range(num_tasks_per_job)]} for _ in range(num_jobs)]
    machines = [f"M{i+1}" for i in range(num_machines)]
    return jobs, machines

# Generate larger problem instances
problem_instances = [generate_problem_instance(20, 10, 5) for _ in range(10)]

# Transport times between machines (constant for simplicity)
transport_times = {
    (f"M{i+1}", f"M{j+1}"): random.randint(1, 5) if i != j else 0
    for i in range(10) for j in range(10)
}

# Q-Learning Parameters
ALPHA = 0.1  # Learning rate
GAMMA = 0.9  # Discount factor
EPSILON = 0.2  # Exploration rate

# Encode solution
def encode_solution(jobs, machines):
    chromosome = []
    machine_end_times = {machine: 0 for machine in machines}
    for job_id in range(len(jobs)):
        job_end_time = 0
        previous_machine = None
        for task_id in range(len(jobs[job_id]["tasks"])):
            machine_id = random.choice(machines)  # Introduce randomness for diversity
            transport_time = transport_times.get((previous_machine, machine_id), 0) if previous_machine else 0
            start_time = max(machine_end_times[machine_id], job_end_time + transport_time)
            task_duration = jobs[job_id]["tasks"][task_id]
            end_time = start_time + task_duration
            machine_end_times[machine_id] = end_time
            job_end_time = end_time
            previous_machine = machine_id
            chromosome.append((job_id, task_id, machine_id, start_time))
    return chromosome

# Fitness function
def calculate_fitness(chromosome, jobs, machines):
    machine_end_times = {machine: 0 for machine in machines}
    job_end_times = [0] * len(jobs)
    penalty = 0

    for i, (job_id, task_id, machine_id, start_time) in enumerate(chromosome):
        job_id = int(job_id)  # Ensure job_id is an integer
        task_id = int(task_id)  # Ensure task_id is an integer
        task_duration = jobs[job_id]["tasks"][task_id]
        if start_time < machine_end_times[machine_id]:
            penalty += (machine_end_times[machine_id] - start_time) * 10

        transport_time = 0
        if i > 0:
            _, _, prev_machine, _ = chromosome[i - 1]
            transport_time = transport_times.get((prev_machine, machine_id), 0)
        end_time = start_time + task_duration + transport_time
        job_end_times[job_id] = max(job_end_times[job_id], end_time)
        machine_end_times[machine_id] = max(machine_end_times[machine_id], end_time)

    makespan = max(job_end_times)
    fitness = makespan + penalty * 0.1  # Scale down penalty
    return fitness

# Q-Learning-based Repair Solution
def q_learning_repair(chromosome, jobs, machines):
    q_table = np.zeros((len(jobs), len(machines)))

    for job_id, task_id, machine_id, start_time in chromosome:
        state = job_id
        action = machines.index(machine_id)

        # Q-Learning update
        reward = -calculate_fitness(chromosome, jobs, machines)  # Negative fitness as reward
        next_state = (state + 1) % len(jobs)
        next_action = np.argmax(q_table[next_state]) if random.random() > EPSILON else random.randint(0, len(machines) - 1)

        q_table[state, action] = q_table[state, action] + ALPHA * (
            reward + GAMMA * np.max(q_table[next_state]) - q_table[state, action]
        )

        # Update chromosome with best action from Q-Learning
        machine_id = machines[next_action]
        chromosome[job_id] = (job_id, task_id, machine_id, start_time)

    return chromosome

# Genetic algorithm with Q-Learning
def genetic_algorithm_with_q_learning(jobs, machines, population_size=50, generations=100):
    population = [encode_solution(jobs, machines) for _ in range(population_size)]
    mutation_rate = 0.2  # Higher initial mutation rate to promote exploration

    for generation in range(generations):
        fitness_values = [calculate_fitness(ind, jobs, machines) for ind in population]
        best_fitness = min(fitness_values)
        best_solution = population[fitness_values.index(best_fitness)]
        average_fitness = sum(fitness_values) / len(fitness_values)
        print(f"Generation {generation + 1}: Best Fitness = {best_fitness}, Average Fitness = {average_fitness:.2f}")

        # Apply Q-Learning to improve solutions
        population = [q_learning_repair(ind, jobs, machines) for ind in population]

        # Selection: Tournament selection
        selected_parents = random.choices(population, k=len(population))

        # Crossover
        offspring = []
        for i in range(0, len(selected_parents) - 1, 2):
            parent1, parent2 = selected_parents[i], selected_parents[i + 1]
            crossover_point1 = random.randint(1, len(parent1) // 2)
            crossover_point2 = random.randint(len(parent1) // 2, len(parent1) - 1)
            child1 = parent1[:crossover_point1] + parent2[crossover_point1:crossover_point2] + parent1[crossover_point2:]
            child2 = parent2[:crossover_point1] + parent1[crossover_point1:crossover_point2] + parent2[crossover_point2:]
            offspring.append(q_learning_repair(child1, jobs, machines))
            offspring.append(q_learning_repair(child2, jobs, machines))

        # Mutation
        for i in range(len(offspring)):
            if random.random() < mutation_rate:  # Dynamic mutation rate
                idx = random.randint(0, len(offspring[i]) - 1)
                job_id, task_id, _, start_time = offspring[i][idx]
                new_machine = random.choice(machines)
                offspring[i][idx] = (int(job_id), int(task_id), new_machine, start_time)
                offspring[i] = q_learning_repair(offspring[i], jobs, machines)

        # Update population
        population = offspring

        # Adjust mutation rate dynamically
        mutation_rate = max(0.05, mutation_rate * 0.95)  # Gradually reduce mutation rate

    return best_solution

# Example usage for problem instances
for idx, (jobs, machines) in enumerate(problem_instances):
    print(f"\nTesting Problem Instance {idx + 1} with Q-Learning")
    best_solution = genetic_algorithm_with_q_learning(jobs, machines, population_size=50, generations=100)
    print("Best Schedule (Timeline):")
    for job_id, task_id, machine_id, start_time in best_solution:
        print(f"Job {job_id + 1}, Task {task_id + 1} on {machine_id} starts at {start_time}")



Testing Problem Instance 1 with Q-Learning
Generation 1: Best Fitness = 378.0, Average Fitness = 475.86
Generation 2: Best Fitness = 1972.0, Average Fitness = 4329.02
Generation 3: Best Fitness = 2200.0, Average Fitness = 4333.02
Generation 4: Best Fitness = 2623.0, Average Fitness = 4525.90
Generation 5: Best Fitness = 2823.0, Average Fitness = 4735.46
Generation 6: Best Fitness = 2901.0, Average Fitness = 4764.36
Generation 7: Best Fitness = 3041.0, Average Fitness = 4939.66
Generation 8: Best Fitness = 2801.0, Average Fitness = 4805.96
Generation 9: Best Fitness = 2799.0, Average Fitness = 4707.42
Generation 10: Best Fitness = 2603.0, Average Fitness = 4882.12
Generation 11: Best Fitness = 2990.0, Average Fitness = 4840.16
Generation 12: Best Fitness = 2579.0, Average Fitness = 4918.02
Generation 13: Best Fitness = 2541.0, Average Fitness = 4777.58
Generation 14: Best Fitness = 3242.0, Average Fitness = 5073.42
Generation 15: Best Fitness = 3320.0, Average Fitness = 4618.66
Generat